# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
from time import time

# import nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.multioutput import MultiOutputClassifier


from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_table', con=engine)

X = df['message'] # reads message column
y = df.iloc[:,4:] # reads all rows, all column starting at [4] ->

In [28]:
# Display message column
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [29]:
# Display categories dataframe made from ETL pipeline
y.head()

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Displays length of characters in each message
df['message'] = df['message'].apply(lambda x: len(x))
df['message']

0         66
1         39
2         31
3        100
4         63
        ... 
26210    148
26211    136
26212    141
26213    191
26214    209
Name: message, Length: 26215, dtype: int64

In [31]:
# Index of column names
y.columns

Index(['related', 'request', 'offer', 'aid related', 'medical help',
       'medical products', 'search and rescue', 'security', 'military',
       'child alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing people', 'refugees', 'death', 'other aid',
       'infrastructure related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid centers', 'other infrastructure',
       'weather related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other weather', 'direct report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [32]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# function to detect url, normalize, tokenize, remove stopwords, lemmatize words 
def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detect urls with url_regex
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalizes and tokenizes (removes punctuations and converts to lowercase)
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # lemmatize and remove stop words
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stopwords.words("english"):
            clean_tokens.append(clean_tok)
    pass
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
# Pipeline with RandomForestClf
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
# Splits dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [35]:
# Fits pipeline on train set 
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [36]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text, lemmatizer=<WordNetLemmatizer>

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [37]:
# Predict pipeline X_test (long process)
y_pred = pipeline.predict(X_test)

In [38]:
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      3998
               request       0.85      0.49      0.62       891
                 offer       0.00      0.00      0.00        24
           aid related       0.76      0.68      0.72      2164
          medical help       0.67      0.08      0.15       435
      medical products       0.71      0.07      0.13       279
     search and rescue       0.69      0.08      0.14       136
              security       0.33      0.01      0.02        96
              military       0.59      0.06      0.11       158
           child alone       0.00      0.00      0.00         0
                 water       0.85      0.44      0.58       335
                  food       0.87      0.61      0.72       584
               shelter       0.80      0.43      0.56       468
              clothing       0.89      0.11      0.20        70
                 money       0.75      

### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
# Pipeline 2 with RFC
pipeline2 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

In [40]:
from sklearn.model_selection import GridSearchCV

# Parameters used for GridsearchCV
parameters2 = {'clf__estimator__n_estimators': [10, 30],}

cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters2, verbose=3)

In [41]:
cv2.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] clf__estimator__n_estimators=10 .................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] ..... clf__estimator__n_estimators=10, score=0.245, total= 7.3min
[CV] clf__estimator__n_estimators=10 .................................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.2min remaining:    0.0s
[CV] ..... clf__estimator__n_estimators=10, score=0.247, total= 7.2min
[CV] clf__estimator__n_estimators=10 .................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 14.5min remaining:    0.0s
[CV] ..... clf__estimator__n_estimators=10, score=0.253, total= 7.2min
[CV] clf__estimator__n_estimators=10 .................................
[CV] ..... clf__estimator__n_estimators=10, score=0.238, total= 7.2min
[CV] clf__estimator__n_estimators=10 .................................
[CV] ..... clf__estimator__n_estimators=10, score=0.253, tot

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 30]}, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:
y_pred2 = cv2.predict(X_test)

In [43]:
print(classification_report(y_test, y_pred2, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.88      3998
               request       0.84      0.46      0.60       891
                 offer       0.00      0.00      0.00        24
           aid related       0.77      0.66      0.71      2164
          medical help       0.73      0.11      0.20       435
      medical products       0.69      0.07      0.13       279
     search and rescue       0.73      0.08      0.15       136
              security       0.25      0.01      0.02        96
              military       0.69      0.07      0.13       158
           child alone       0.00      0.00      0.00         0
                 water       0.85      0.28      0.42       335
                  food       0.86      0.55      0.67       584
               shelter       0.83      0.32      0.47       468
              clothing       0.68      0.19      0.29        70
                 money       0.70      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

# Pipeline 3 with AdaboostClf
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [54]:
# tuning parameters

parameters3 = {'clf__estimator__n_estimators': [10, 40], 
               'clf__estimator__learning_rate': [1,2]}


cv3 = GridSearchCV(estimator=pipeline3, param_grid=parameters3, verbose=3)

In [55]:
cv3

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [1, 2],
                         'clf__estimator__n_estimators': [10, 40]},
             verbose=3)

In [56]:
cv3.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, score=0.207, total= 9.2min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.2min remaining:    0.0s
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, score=0.198, total= 9.9min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 19.1min remaining:    0.0s
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, score=0.212, total= 9.9min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, score=0.209, total=10.3min
[CV] clf__estimator

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024AF338C5E0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [1, 2],
                         'clf__estimator__n_estimators': [10, 40]},
             verbose=3)

In [58]:
y_pred3 = cv3.predict(X_test)

In [59]:
# cv3 best parameters
print(classification_report(y_test, y_pred3, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      3998
               request       0.79      0.47      0.59       891
                 offer       0.00      0.00      0.00        24
           aid related       0.77      0.56      0.65      2164
          medical help       0.66      0.24      0.35       435
      medical products       0.69      0.31      0.43       279
     search and rescue       0.58      0.15      0.24       136
              security       0.19      0.03      0.05        96
              military       0.57      0.25      0.35       158
           child alone       0.00      0.00      0.00         0
                 water       0.76      0.65      0.70       335
                  food       0.86      0.65      0.74       584
               shelter       0.75      0.48      0.59       468
              clothing       0.69      0.47      0.56        70
                 money       0.53      

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv3, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.